In [43]:
__file__ = "__init__.py"

In [44]:
import sys, os, json5, glob, warnings
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, random_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, EarlyStoppingCallback
import torch.nn as nn
import evaluate

warnings.filterwarnings("ignore")
project_root = Path(__file__).resolve().parents[1]
sys.path.append(str(project_root))

from utils.qgene import generate_text

paths = {
    "processed": os.path.abspath(f"{project_root}/data/storage/processed"),
    "qfragments": os.path.abspath(f"{project_root}/intents/qfragments.json"),
    "questions": os.path.abspath(f"{project_root}/intents/questions.csv"),
    "qtrain": os.path.abspath(f"{project_root}/intents/qtrain.csv"),
    "models": os.path.abspath(f"{project_root}/models/t5-small"),
    "results": os.path.abspath(f"{project_root}/training/results"),
}

In [45]:
class QTrainDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_source_length=512, max_target_length=128):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        # Input: "Extract component: " + question
        input_text = "Extract component: " + row["question"]
        # Target: chuỗi các component theo định dạng cố định
        target_text = (
            f"BRAND: {row['brand']}; PRICE: {row['price']}; RAM: {row['ram']}; "
            f"GPU: {row['gpu']}; CPU: {row['cpu']}; DISPLAY: {row['display']}; "
            f"REFRESH_RATE: {row['refresh rate']}"
            )
        source = self.tokenizer.encode_plus(
            input_text,
            max_length=self.max_source_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        target = self.tokenizer.encode_plus(
            target_text,
            max_length=self.max_target_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        source_ids = source["input_ids"].squeeze()              # [max_source_length]
        source_mask = source["attention_mask"].squeeze()        # [max_source_length]
        target_ids = target["input_ids"].squeeze()              # [max_target_length]
        # Đặt các token pad thành -100 để không tính loss cho chúng
        target_ids[target_ids == self.tokenizer.pad_token_id] = -100
        
        return {
            "input_ids": source_ids,
            "attention_mask": source_mask,
            "labels": target_ids
        }

In [46]:
def integrate_custom_embeddings(model, custom_emb_dim=256):
    """
    Tích hợp custom embeddings vào mô hình T5.
    Giả sử bạn có ma trận embedding tùy chỉnh (ở đây demo bằng random) với shape (vocab_size, custom_emb_dim).
    Sau đó, dùng một lớp projection (Linear) để chuyển từ custom_emb_dim (256) lên d_model của T5 (512).
    """
    vocab_size = model.config.vocab_size
    # Ví dụ: ma trận embedding tùy chỉnh (có thể thay bằng mô hình word2vec/fastText tự huấn luyện)
    custom_embedding_matrix = torch.randn(vocab_size, custom_emb_dim)
    projection_layer = nn.Linear(custom_emb_dim, model.config.d_model)
    with torch.no_grad():
        projected_embeddings = projection_layer(custom_embedding_matrix)
        # Cập nhật embedding chung của T5
        model.shared.weight.copy_(projected_embeddings)
    return model

In [47]:
def compute_metrics(eval_pred):
    """
    Hàm compute_metrics sử dụng evaluate để tính ROUGE và BLEU.
    eval_pred là tuple (predictions, labels).
    """
    predictions, labels = eval_pred

    # Giải mã dự đoán và nhãn
    decoded_preds = global_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, global_tokenizer.pad_token_id)
    decoded_labels = global_tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Tính toán ROUGE
    rouge_metric = evaluate.load("rouge")
    rouge_output = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    # Tính toán BLEU: tokenize các chuỗi đầu ra
    bleu_metric = evaluate.load("bleu")
    tokenized_preds = [pred.split() for pred in decoded_preds]
    tokenized_refs = [[ref.split()] for ref in decoded_labels]
    bleu_output = bleu_metric.compute(predictions=tokenized_preds, references=tokenized_refs)
    
    result = {
        "rouge1": rouge_output.get("rouge1", 0.0),
        "rouge2": rouge_output.get("rouge2", 0.0),
        "rougeL": rouge_output.get("rougeL", 0.0),
        "bleu": bleu_output.get("bleu", 0.0)
    }
    return result

In [48]:
class NLPModel:
    def __init__(self, datanum: int = 1000):
        # Xác định thư mục gốc của dự án
        project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
        self.paths = {
            "qtrain": os.path.join(project_root, "intents", "qtrain.csv"),
            "results": os.path.join(project_root, "training", "results"),
            "models": os.path.join(project_root, "models", "t5-custom")
        }
        os.makedirs(self.paths["results"], exist_ok=True)
        os.makedirs(self.paths["models"], exist_ok=True)
        
        # Cập nhật dataset: nếu file qtrain.csv đã tồn tại thì nối thêm dữ liệu mới từ generate_text
        self.data = self._dataset(datanum)
        
        # Tải tokenizer và mô hình T5 pre-trained
        self.tokenizer = T5Tokenizer.from_pretrained("t5-small")
        self.model = T5ForConditionalGeneration.from_pretrained("t5-small")
        
        # Tích hợp custom embeddings vào mô hình (với custom embedding dimension = 256)
        self.model = integrate_custom_embeddings(self.model, custom_emb_dim=256)
        
        # Xây dựng dataset
        self.dataset = QTrainDataset(csv_file=self.paths["qtrain"], tokenizer=self.tokenizer)
        # Tách dataset thành training (80%) và validation (20%)
        train_size = int(0.8 * len(self.dataset))
        val_size = len(self.dataset) - train_size
        self.train_dataset, self.val_dataset = random_split(self.dataset, [train_size, val_size])
        
        # Thiết lập các tham số huấn luyện với Trainer của HuggingFace
        self.training_args = TrainingArguments(
            output_dir=self.paths["results"],
            num_train_epochs=3,
            per_device_train_batch_size=8,    # batch size nhỏ để tránh OOM
            per_device_eval_batch_size=8,
            gradient_accumulation_steps=4,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            logging_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            save_total_limit=3,
            report_to="none"
        )
        
        # Tạo Trainer (bao gồm EarlyStoppingCallback với patience = 5)
        self.trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.val_dataset,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
        )
    
    def _dataset(self, datanum: int = 1000) -> pd.DataFrame:
        """
        Nếu file qtrain.csv tồn tại, đọc nó và nối thêm dữ liệu mới được tạo ra từ generate_text(num, choice="s1").
        Sau đó lưu lại file qtrain.csv và trả về DataFrame.
        """
        from utils.qgene import generate_text  # Hàm generate_text của bạn
        if os.path.exists(self.paths["qtrain"]):
            old_data = pd.read_csv(self.paths["qtrain"])
            new_questions = pd.concat([old_data, generate_text(datanum, choice="s1")])
        else:
            new_questions = generate_text(datanum, choice="s1")
        new_questions.to_csv(self.paths["qtrain"], index=False)
        return new_questions
    
    def train(self):
        """ Huấn luyện mô hình sử dụng Trainer của HuggingFace. """
        self.trainer.train()
        self.model.save_pretrained(self.paths["models"])
        self.tokenizer.save_pretrained(self.paths["models"])
    
    def inference(self, input_text: str) -> dict:
        input_text = "Extract component: " + input_text
        inputs = self.tokenizer.encode_plus(
            input_text,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512
        )
        # Chuyển các tensor đầu vào sang cùng thiết bị với model
        device = next(self.model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        outputs = self.model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128
        )
        decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        components = ["BRAND", "PRICE", "RAM", "GPU", "CPU", "DISPLAY", "REFRESH_RATE"]
        result = {comp.lower(): None for comp in components}
        for part in decoded.split(";"):
            if ":" in part:
                key, value = part.split(":", 1)
                key = key.strip().upper()
                value = value.strip() if value.strip() else None
                if key in components:
                    result[key.lower()] = value
        return result

In [49]:
model_instance = NLPModel(datanum=10000) 

In [50]:
model_instance.train()

Epoch,Training Loss,Validation Loss


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
sample_text = "recommend me a laptop with RTX 3070, 16GB RAM, around 2000 USD"
inference_result = model_instance.inference(sample_text)
print("Inference result:", inference_result)

Inference result: {'brand': None, 'price': None, 'ram': None, 'gpu': None, 'cpu': None, 'display': None, 'refresh_rate': None}
